In [4]:
!pip install langchain #For numbers of task provide extract context to llm
!pip install pypdf #read pdf document and extract information
!pip install sentence_transformers #Embedding
! pip install chromadb #Store Text data and embedding
!pip install accelerate
!pip install --upgrade accelerate
%pip install bitsandbytes

In [5]:
from langchain.docstore.document import Document
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
loader = PyPDFLoader('/content/drive/MyDrive/Dataset.pdf')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=400, chunk_overlap=40)
documents = text_splitter.split_documents(documents)

In [8]:
from sentence_transformers import SentenceTransformer, models

In [9]:
hf_embed = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

Vector_db = Chroma.from_documents(collection_name="document_docs", documents=documents, embedding=hf_embed, persist_directory="/content/drive/MyDrive/Conv_bot/Data/Docs/Persist_dir")
Vector_db.similarity_search("dummy") # tickle it to persist metadata (?)
Vector_db.persist()
pdf_vector_db_path = "/content/drive/MyDrive/Conversational_bot/Data/Docs/db_path"

hf_embed = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
db = Chroma(collection_name="document_docs", embedding_function=hf_embed, persist_directory=pdf_vector_db_path)

In [10]:
def get_similar_docs(question, similar_doc_count):
  return Vector_db.similarity_search(question, k=similar_doc_count)
for doc in get_similar_docs("lump under left nipple and stomach pain (male) Hi,I have recently noticed a few weeks ago a lump under my nipple, it hurts to touch and is about the size of a quarter. Also I have bern experiencing stomach pains that prevent me from eating. I immediatly feel full and have extreme pain. Please help",2):
  print(doc.page_content)

148 
 PEPTIC ULCER DISEASE  
 
Gastric and duodenal ulcers usually cannot be differentiated based on history alone, although some 
findings may be suggestive. Epigastric pain is the most common symptom of both ga stric and duodenal 
ulcers. It is characterized by a gnawing or burning sensation and occurs after meals —classically, shortly 
after meals with gastric ulcer and 2 -3 hours afterward with duodenal ulcer.  
 
Possible manifestations include the following:  
 Dyspeps ia, including belching, bloating, distention, and fatty food intolerance  
 Heartburn  
 Chest discomfort  
 Hematemesis or melena resulting from gastrointestinal bleeding. Melena may be intermittent 
over several days or multiple episodes in a single day.  
 Rarel y, a briskly bleeding ulcer can present as hematochezia.  
 Symptoms consistent with anemia (eg, fatigue, dyspnea) may be present  
 Sudden onset of symptoms may indicate perforation.  
 NSAID -induced gastritis or ulcers may be silent, especiall

In [11]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
from langchain import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.chains.question_answering import load_qa_chain

In [12]:
template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

  Instruction:
  if you are a doctor please provide reasonable causes for the following symptoms.

  {context}

  Question: {question}

  Response:
  """
def build_qa_chain():
  torch.cuda.empty_cache()
  model_name = "databricks/dolly-v2-3b" # can use dolly-v2-3b or dolly-v2-7b for smaller model and faster inferences.

  device = "cuda:0" if torch.cuda.is_available() else "cpu"
  # Increase max_new_tokens for a longer response
  # Other settings might give better results! Play around
  instruct_pipeline = pipeline(model=model_name, torch_dtype=torch.bfloat16, trust_remote_code=True, device_map="auto", return_full_text=True, max_new_tokens=256, top_p=0.95, top_k=50,model_kwargs={'load_in_8bit': False})
  model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16, trust_remote_code=True)

  prompt = PromptTemplate(input_variables=['context', 'question'], template=template)

  hf_pipe = HuggingFacePipeline(pipeline=instruct_pipeline)
  return load_qa_chain(llm=hf_pipe, chain_type="stuff", prompt=prompt, verbose=True)# Note: if you use dolly 12B or smaller model but a GPU with less than 24GB RAM, use 8bit. This requires %pip install bitsandbytes
  # instruct_pipeline = pipeline(model=model_name, trust_remote_code=True, device_map="auto", model_kwargs={'load_in_8bit': True})

In [13]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'



In [14]:
import torch
from transformers import pipeline
!pip install accelerate
!pip install -i https://test.pypi.org/simple/bitsandbytes
qa_chain = build_qa_chain()

ERROR: You must give at least one requirement to install (see "pip help install")


In [18]:
def displayHTML(html):
    """Display HTML in Jupyter notebook."""
    from IPython.display import HTML
    display(HTML(html))

In [16]:
def answer_question(question):
  similar_docs = get_similar_docs(question, similar_doc_count=3)
  result = qa_chain({"input_documents": similar_docs, "question": question})
  result_html = f"<p><blockquote style=\"font-size:24\">{question}</blockquote></p>"
  result_html += f"<p><blockquote style=\"font-size:18px\">{result['output_text']}</blockquote></p>"
  result_html += "<p><hr/></p>"
  for d in result["input_documents"]:
    source_id = d.metadata["source"]
    result_html += f"<p><blockquote>{d.page_content}<br/>(Source: <a href=\"https://gardening.stackexchange.com/a/{source_id}\">{source_id}</a>)</blockquote></p>"
  displayHTML(result_html)

In [17]:
answer_question("I am feeling dizzy. Can you please tell me what are the causes ")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

  Instruction:
  if you are a doctor please provide reasonable causes for the following symptoms.

  230 
 Diagnosis  
Tension -type headache, migraine (with or without aura) and cluster headache  
 Diagnose tension -type headache, migraine or cluster headache according to the headache 
features in the table  
Migraine with aura  
 Suspect aura in people who present with or without hea dache and with neurological symptoms 
that:  
o are fully reversible and 
o develop gradually, either alone or in succession, over at least 5 minutes and 
o last for 5 –60 minutes  
 Diagnose migraine with aura in people who present with or without headache and with one or 
more of the following typical aura symptoms that meet the criteria (see above):  
o visual symptoms that may be positive